#### Import libraries

In [1]:
import numpy as np
import os
from PIL import Image

#### Training data - Bears

In [2]:
train_bears_dir = r"pandas-bears\1. train\bears"
train_bears_imgs = []
for img in os.listdir(train_bears_dir):
    img_path = os.path.join(train_bears_dir, img)
    train_bears_img = Image.open(img_path, 'r')
    train_bears_imgs.append(train_bears_img)

train_bears_data = np.array(train_bears_imgs)

#### Training data - Pandas

In [3]:
train_pandas_dir = r"pandas-bears\1. train\pandas"
train_pandas_imgs = []
for img in os.listdir(train_pandas_dir):
    img_path = os.path.join(train_pandas_dir, img)
    train_pandas_img = Image.open(img_path, 'r')
    train_pandas_imgs.append(train_pandas_img)

train_pandas_data = np.array(train_pandas_imgs)

#### Testing data - Bears

In [4]:
test_bears_dir = r"pandas-bears\2. test\bears"
test_bears_imgs = []
for img in os.listdir(test_bears_dir):
    img_path = os.path.join(test_bears_dir, img)
    test_bears_img = Image.open(img_path, 'r')
    test_bears_imgs.append(test_bears_img)

test_bears_data = np.array(test_bears_imgs)

#### Testing data - Pandas

In [5]:
test_pandas_dir = r"pandas-bears\2. test\pandas"
test_pandas_imgs = []
for img in os.listdir(test_pandas_dir):
    img_path = os.path.join(test_pandas_dir, img)
    test_pandas_img = Image.open(img_path, 'r')
    test_pandas_imgs.append(test_pandas_img)

test_pandas_data = np.array(test_pandas_imgs)

#### Combine

In [19]:
X_train = np.concatenate((train_pandas_data, train_bears_data), axis = 0)
X_test = np.concatenate((test_pandas_data, test_bears_data), axis = 0)

X_train = X_train / X_train.max()
X_test = X_test / X_test.max()

_Let Pandas be 0 & Bears be 1._

In [20]:
train_lab_p = np.zeros(train_pandas_data.shape[0])
train_lab_b = np.ones(train_bears_data.shape[0])

test_lab_p = np.zeros(test_pandas_data.shape[0])
test_lab_b = np.zeros(test_bears_data.shape[0])

In [21]:
y_train = np.concatenate((train_lab_p, train_lab_b), axis = 0)
y_test = np.concatenate((test_lab_p, test_lab_b), axis = 0)

In [22]:
print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

Training data shape: (500, 256, 256, 3)
Testing data shape: (100, 256, 256, 3)


#### Convert np array to torch tensor

In [23]:
import torch

In [24]:
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)

y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

#### Model definitions & stuff

In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [26]:
class CNN(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 100)
        self.fc2 = nn.Linear(100, 1)
    
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.sigmoid(self.fc2(x))
        return x

cnn = CNN()

In [27]:
import torch.optim as optim

In [28]:
criterion = nn.BCELoss()
optimizer = optim.Adam(cnn.parameters(), lr = 0.001)

#### Model training

In [29]:
from torch.utils.data import DataLoader

In [30]:
X_train = DataLoader(X_train.float())
X_test = DataLoader(X_test)

y_train = DataLoader(y_train.float())
y_test = DataLoader(y_test)

In [31]:
for epoch in range(10):

    running_loss = 0.0
    for i in range(len(X_train)):

        optimizer.zero_grad()

        outputs = cnn(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        print(running_loss)

TypeError: conv2d() received an invalid combination of arguments - got (DataLoader, Parameter, Parameter, tuple, tuple, tuple, int), but expected one of:
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, tuple of ints padding = 0, tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!DataLoader!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
 * (Tensor input, Tensor weight, Tensor bias = None, tuple of ints stride = 1, str padding = "valid", tuple of ints dilation = 1, int groups = 1)
      didn't match because some of the arguments have invalid types: (!DataLoader!, !Parameter!, !Parameter!, !tuple of (int, int)!, !tuple of (int, int)!, !tuple of (int, int)!, !int!)
